<a href="https://colab.research.google.com/github/royanlord/DL_Image-Classification_Pneumonia/blob/main/DL_Pneumonia_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# ENVIRONMENT SETUP
# =========================================================
ENV = "colab"
# pilih salah satu: "colab" atau "kaggle"

import os


In [2]:
if ENV == "colab":
    from google.colab import files
    files.upload()  # upload kaggle.json


Saving kaggle.json to kaggle.json


In [3]:
if ENV == "colab":
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json


🔷 3. DOWNLOAD DATASET (AMAN & OTOMATIS)

A. Kaggle Chest X-ray Pneumonia (Baseline)

In [5]:
if ENV == "colab":
    !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
    !unzip -q chest-xray-pneumonia.zip
    PNEUMONIA_DIR = "/content/chest_xray"
else:
    PNEUMONIA_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray"


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 96% 2.21G/2.29G [00:22<00:03, 26.4MB/s]
100% 2.29G/2.29G [00:22<00:00, 109MB/s] 


B. NIH ChestX-ray8 (Generalisasi)

In [ ]:
if ENV == "colab":
    !kaggle datasets download -d nih-chest-xrays/data
    !unzip -q data.zip -d nih
    NIH_DIR = "/content/nih"
else:
    NIH_DIR = "/kaggle/input/nih-chest-xrays"


Dataset URL: https://www.kaggle.com/datasets/nih-chest-xrays/data
License(s): CC0-1.0
^C
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [6]:
print("PNEUMONIA_DIR:", PNEUMONIA_DIR)
# Path Dataset (pastikan sesuai dengan struktur setelah unzip)
BASE_DIR = 'chest_xray'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

# Pemeriksaan Data
print(f"\nDataset Train Normal: {len(os.listdir(os.path.join(TRAIN_DIR, 'NORMAL')))} citra")
print(f"Dataset Train Pneumonia: {len(os.listdir(os.path.join(TRAIN_DIR, 'PNEUMONIA')))} citra")
# print("NIH_DIR:", NIH_DIR)

assert os.path.exists(PNEUMONIA_DIR), "❌ Pneumonia dataset not found!"
# assert os.path.exists(NIH_DIR), "❌ NIH dataset not found!"


PNEUMONIA_DIR: /content/chest_xray

Dataset Train Normal: 1341 citra
Dataset Train Pneumonia: 3875 citra


In [7]:
import cv2, random
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.applications import (
    Xception,
    EfficientNetB0,
    EfficientNetV2L
)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

from sklearn.metrics import classification_report, confusion_matrix


In [8]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)


In [9]:
def apply_clahe(image, clip_limit=2.0, grid_size=(8,8)):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(
        clipLimit=clip_limit,
        tileGridSize=grid_size
    )
    image = clahe.apply(image)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    return image


In [10]:
def clahe_preprocess(image, params=None):
    if params is None:
        return image
    clip, grid = params
    return apply_clahe(image, clip_limit=clip, grid_size=(grid, grid))


In [11]:
def load_pneumonia_dataset(img_size=(224,224), batch_size=16):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        f"{PNEUMONIA_DIR}/train",
        image_size=img_size,
        batch_size=batch_size,
        label_mode="binary"
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        f"{PNEUMONIA_DIR}/val",
        image_size=img_size,
        batch_size=batch_size,
        label_mode="binary"
    )
    return train_ds, val_ds


In [ ]:
def load_nih_binary(csv_path, img_dir):
    df = pd.read_csv(csv_path)

    df["target"] = df["Finding Labels"].apply(
        lambda x: 1 if "Pneumonia" in x else 0
    )

    return df


In [12]:
def build_model(backbone="efficientnetb0", num_classes=1):

    if backbone == "xception":
        base = Xception(weights="imagenet", include_top=False)
    elif backbone == "efficientnetb0":
        base = EfficientNetB0(weights="imagenet", include_top=False)
        base.trainable = False
    elif backbone == "efficientnetv2l":
        base = EfficientNetV2L(weights="imagenet", include_top=False)
    else:
        raise ValueError("Unknown backbone")

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(num_classes, activation="sigmoid")(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer=Adam(1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", AUC(name="auc")]
    )

    return model


In [13]:
def train_model(model, train_ds, val_ds, epochs=10):
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )
    return history


In [14]:
# Load the datasets and make them global for clahe_objective to access
global train_ds, val_ds
train_ds, val_ds = load_pneumonia_dataset()

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


TRAIN DENGAN SUBSET KECIL

In [15]:
def get_subset_dataset(dataset, max_batches=10):
    return dataset.take(max_batches)

In [16]:
train_ds_small = get_subset_dataset(train_ds, max_batches=10)
val_ds_small   = get_subset_dataset(val_ds, max_batches=5)

In [17]:
def clahe_fitness(params):
    clip = params[0]
    grid = int(params[1])

    # train model dengan subset kecil
    model = build_model("efficientnetb0")
    # history = train_model(model, train_ds, val_ds, epochs=3)
    history = train_model(model, train_ds_small, val_ds_small, epochs=3)

    return -max(history.history["val_auc"])


🧩 9. XAI (PLACEHOLDER TERSTRUKTUR)

In [ ]:
def generate_gradcam(model, image):
    """
    Placeholder: Grad-CAM implementation
    Akan dipanggil setelah model final dipilih
    """
    pass


🧩 10. IMPLEMENTASI OPTIMASI CLAHE



A. Differential Evolution

In [18]:
from scipy.optimize import differential_evolution

def clahe_objective(params):
    clip = params[0]
    grid = int(params[1])

    model = build_model("efficientnetb0")
    history = model.fit(
        train_ds_small,
        validation_data=val_ds_small,
        # epochs=3,
        epochs=1,
        verbose=0
    )

    return -max(history.history["val_auc"])


In [ ]:
# bounds = [(1.0, 5.0), (4, 16)]
bounds = [(1.0, 4.0), (6, 12)]

result_de = differential_evolution(
    clahe_objective,
    bounds,
    # maxiter=5,
    # popsize=6,
    maxiter=3,
    popsize=4,
    disp=True
)

best_clahe_de = result_de.x
print("Best CLAHE (DE):", best_clahe_de)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
differential_evolution step 1: f(x)= -0.8828125
differential_evolution step 2: f(x)= -0.8828125


B. SIMULATED ANNEALING

In [ ]:
import math

def simulated_annealing(objective, init, temp=1.0, cooling=0.95, steps=20):
    current = init
    current_score = objective(current)
    best = current

    for _ in range(steps):
        neighbor = [
            np.clip(current[0] + np.random.uniform(-0.5, 0.5), 1.0, 5.0),
            int(np.clip(current[1] + np.random.randint(-2, 3), 4, 16))
        ]

        score = objective(neighbor)

        if score < current_score or np.exp((current_score - score) / temp) > np.random.rand():
            current, current_score = neighbor, score
            best = neighbor

        temp *= cooling

    return best


In [ ]:
best_clahe_sa = simulated_annealing(
    clahe_objective,
    init=best_clahe_de
)

print("Best CLAHE (SA):", best_clahe_sa)
